## Loading packages, constants, and functions

In [1]:
import functions as f
import constants as c
import classes as cl

import pandas as pd
import os, joblib, time

from netwulf import visualize
print(f"Current directories in {c.cwd}\n {os.listdir()}")

long = c.cwd + "\\raw_data\\long_alzheimers.tsv"
short = c.cwd + "\\raw_data\\short_alzheimers.tsv"

Current directories in /Users/asbjornhansen/GitRep/02807_project_Group13-3\
 ['joblib_vars', 'functions.py', 'modularity.ipynb', 'Notebook.ipynb', 'constants.py', 'raw_data', '__pycache__', 'README.md', '.gitignore', 'OldCode_saveforReport.ipynb', 'enrichment.ipynb', 'classes.py', '.git']


## Loading Datasets

#### Creating id_prot and saving
id_prot is a dictionary meant to point a id <integer> to a protein name <string> with 19 str characters \
id_prot is saved using joblib.dump() in dir "joblib_vars", and can be reloaded as a dict object using joblib.load(path)

In [ ]:
chunksize = 10
info_df = pd.read_csv("https://stringdb-downloads.org/download/protein.info.v12.0/9606.protein.info.v12.0.txt.gz", compression='gzip', sep="\t")
info_df = info_df[["#string_protein_id"]].values
id_prot = dict()
for c, e in enumerate(info_df):
    id_prot[c] = e[0]
print(id_prot)
joblib.dump(id_prot, "./joblib_vars/id_prot.joblib")

#### Modifying raw datasets
modified datasets are stored in "/mod_data/" for easy retrieval

In [ ]:
"""
An example of an interaction dataset loaded
"""
alz_int_df = pd.read_csv("./raw_data/long_alzheimers.tsv", sep="\t")
alz_int_df.columns = ["Prot1_ShortName", "Prot2_ShortName", "protein1", "protein2", "col1", "col2", "col3", "col4", "col5", "col6", "col7", "col8", "col9"]
alz_int_df

In [ ]:
HS_int_df = pd.read_csv("https://stringdb-downloads.org/download/protein.links.detailed.v12.0/9606.protein.links.detailed.v12.0.txt.gz", compression="gzip", sep=" ")
HS_int_df = HS_int_df[HS_int_df["experimental"] > 0] #removing interactions with no experimental relevance 
#joblib.dump(HS_int_df, "./joblib_vars/HS_int_df") #FILE TOO LARGE 
HS_int_df


Creating HS_int_simple, containing protein1 and protein2 names converted to int using id_prot. \
If all interactions are weight=1, only the names (ids) of the proteins are needed when constructing the interaction network

In [1]:
id_prot = joblib.load("./joblib_vars/id_prot.joblib")

#Swapping key value in dict
id_swap = dict()
for k, v in id_prot.items():
    id_swap[str(v)] = k
print(id_swap)

#exporting modified HS_int
HS_int_simple = HS_int_df.map(lambda x: id_swap[x] if x in id_swap else x).reset_index()
HS_int_simple = HS_int_simple[["protein1", "protein2"]]
joblib.dump(HS_int_simple, "./joblib_vars/HS_int_simple.joblib")


NameError: name 'joblib' is not defined

## Running the class

### Setting the color of the progress bar

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

### Running the class

In [3]:
a = cl.interaction_network(testdataset = True, threshold=0.65)
a.create_encoding_dict()
a.load_data()

Creating encoding table
Fetching data
Cropping data
Given threshold: 0.65 filtered out combined_score values: 310.0 and below


Parsing data:   0%|          | 0/78868 [00:00<?, ?it/s]

Data loaded in var .vertices, with size: 0.147552mb


In [ ]:
#Example code for the format of a.vertices
for root in a.vertices.keys():
    print("root", "neighbor", "norm_score")
    for neighbor in a.vertices[root].keys():
        print(root, neighbor, a.vertices[root][neighbor])
    break

In [4]:
edges_used = a.evaluate_most_used_path(a.vertices)

Parsing encoding_edgesused data:   0%|          | 0/39434 [00:00<?, ?it/s]

In [5]:
edges_used = sorted(edges_used.items(), key = lambda x:x[1], reverse=True)
for i in range(10):
    print(edges_used[i][0], edges_used[i][1])

493 3044591
979 2292854
1444 1919756
4668 1917023
1497 1879340
2434 1724455
2059 1722026
13809 1665673
2249 1628455
7685 1581202


In [ ]:
# Example usage of check_connection
vertexes = {"A": ["B", "C"],
            "B": ["A", "C", "D"],
            "C": ["A", "B"],
            "D": ["B", "E"],
            "E": ["D"],
            "F": ["G"],
            "G": ["F"]}

is_connected, connected_vertices = check_connection(vertexes)
print("The network is fully connected: {}".format(is_connected))
for i, group in enumerate(connected_vertices):
    print("Grouped vertices {}:".format(i+1))
    print(group)